In [ ]:
# Julia code for dynamic curb allocation comparing MIP and DW method (QCP constraint included)
# written by Nawaf
# curb allocation for Seattle (Insignia) without bus stop consideration
using Dates
now_time=now() - Dates.Hour(7)
println("now_time: ", now_time)
start_time=Int(floor((Dates.hour(now_time) * 60 + Dates.minute(now_time)) / 15))
println("start_time: ", start_time) # the start index for a 15 minute block based on the current seattle local time
nb=26; # no of blocks
T=10; # no of timesteps
E=3; # no of curb types (1=paid parking, 2=commercial vehicles, 3=busus/public)

In [ ]:
using CSV
using DataFrames

In [ ]:
# read the curb length data
curb_data=CSV.read("./space_val_data/Iter2/ekey_length.csv", DataFrame)  # reading node data
tot_curbs=sum(curb_data[1:nb,3]);
loc=3; # no of demand locations
tot_T=96; # total time of data available

In [ ]:
# read the space valuation of curbs for buses
curb_value_bus=zeros(Float64,tot_curbs,loc,tot_T);
curb_value_bus_tot=zeros(Float64,tot_curbs,tot_T);
global mp=0;
for i=1:nb
    curb_name=string(curb_data[i,2]);
    for k=1:loc
        loc_name=string(k);
        value_data=CSV.read(string("./space_val_data/Iter2/bus/",curb_name,"_loc_",loc_name,"_bus.csv"), DataFrame; header=false)
        for j=1:curb_data[i,3]
            for p=1:tot_T
                curb_value_bus[mp+j,k,p]=value_data[j,p];
            end
        end
    end
    global mp=mp+curb_data[i,3];
end
curb_value_bus_tot=reshape(sum(curb_value_bus,dims=2),tot_curbs,tot_T);

In [ ]:
# read the space valuation of curbs for paid parking
curb_value_paid=zeros(Float64,tot_curbs,loc,tot_T);
curb_value_paid_tot=zeros(Float64,tot_curbs,tot_T);
global mp=0;
for i=1:nb
    curb_name=string(curb_data[i,2]);
    for k=1:loc
        loc_name=string(k);
        value_data=CSV.read(string("./space_val_data/Iter2/paid/",curb_name,"_loc_",loc_name,"_paid.csv"), DataFrame; header=false)
        for j=1:curb_data[i,3]
            for p=1:tot_T
                curb_value_paid[mp+j,k,p]=value_data[j,p];
            end
        end
    end
    global mp=mp+curb_data[i,3];
end
curb_value_paid_tot=reshape(sum(curb_value_paid,dims=2),tot_curbs,tot_T);

In [ ]:
# read the space valuation of curbs for cv
curb_value_cv=zeros(Float64,tot_curbs,loc,tot_T);
curb_value_cv_tot=zeros(Float64,tot_curbs,tot_T);
global mp=0;
for i=1:nb
    curb_name=string(curb_data[i,2]);
    for k=1:loc
        loc_name=string(k);
        value_data=CSV.read(string("./space_val_data/Iter2/cv/",curb_name,"_loc_",loc_name,"_cv.csv"), DataFrame; header=false)
        for j=1:curb_data[i,3]
            for p=1:tot_T
                curb_value_cv[mp+j,k,p]=value_data[j,p];
            end
        end
    end
    global mp=mp+curb_data[i,3];
end
curb_value_cv_tot=reshape(sum(curb_value_cv,dims=2),tot_curbs,tot_T);

In [ ]:
# read the distance (latitude and longitude) of curbs

dist_curb=zeros(Float64,tot_curbs,2);
global mp=0;
for i=1:nb
    curb_name=string(curb_data[i,2]);
    dist_data=CSV.read(string("./space_val_data/Iter2/distance/",curb_name,"_dist.csv"), DataFrame; header=false)
    for j=1:curb_data[i,3]
        for k=1:2
            dist_curb[mp+j,k]=dist_data[j,k];
        end
    end
    global mp=mp+curb_data[i,3];
end

In [ ]:
# form the distance matrix
dist_matrix=zeros(Float64,tot_curbs,tot_curbs);
for i=1:tot_curbs
    for j=i+1:tot_curbs
        #if i!=j
            dist_matrix[i,j]=(1e-5)/((dist_curb[i,1]-dist_curb[j,1])^2+(dist_curb[i,2]-dist_curb[j,2])^2);
            #dist_matrix[j,i]=(1e-5)/((dist_curb[i,1]-dist_curb[j,1])^2+(dist_curb[i,2]-dist_curb[j,2])^2);
        #else
            #dist_matrix[i,j]=1;
        #end
    end
end

In [ ]:
 b=100; # no of changes in allocations between timesteps
 tmin=[120; 120; 120]; # minimum allocations over time
 tmax=[1500; 1500; 1500]; # maximum allocations over time
t1min=[50; 50; 50]; # minimum allocations at one timestep (of a type of allocation)
t1max=[125; 125; 125]; # maximum allocations at one timestep (of a type of allocation)

In [ ]:
# optimization problem starts
# this is being dynamically set in the first block
#start_time=49;  # start the optimization horizon at this timestep (from tot_T=96)
using JuMP
using SCIP
#using Gurobi
#using Mosek
#using MosekTools

In [ ]:
#JM=Model(with_optimizer(Gurobi.Optimizer, MIPGap=.005, OutputFlag=0)) # silent run
JM=Model(SCIP.Optimizer)#, MIPGap=.01)
#set_optimizer_attribute(JM, "MIPGap", .01);

In [ ]:
# defining variables
@variable(JM,U[1:tot_curbs,1:T,1:E], Bin) # allocation variable
#(1=paid parking, 2=commercial vehicles, 3=busus/public)
@variable(JM,x[1:tot_curbs,1:T-1,1:E], Int) # change in allocation between timesteps
@variable(JM,w[1:T,1:E]) # regularization term for spacing

In [ ]:
# defining objective
@objective(JM,Min,-sum(sum(curb_value_paid_tot[:,start_time+1:start_time+T].*U[:,:,1]))-sum(sum(curb_value_cv_tot[:,start_time+1:start_time+T].*U[:,:,2]))-sum(sum(curb_value_bus_tot[:,start_time+1:start_time+T].*U[:,:,3])));
 # maximize linear revenue+ regularization for spacing (neglect regularization for now)

In [ ]:
# defining constraints
for i=1:tot_curbs
    for t=1:T
        @constraint(JM,sum(U[i,t,:])==1) # at a time and location only one allocation allowed
    end
end
for t=1:T-1
    for i=1:tot_curbs
        for e=1:E
            @constraint(JM,x[i,t,e] .== U[i,t+1,e]-U[i,t,e])
        end
    end
    @constraint(JM,sum(sum(x[:,t,:].^2))<=2*b) # limiting change in allocation between timesteps
end

In [ ]:
# maximum and minimum no of allocations over time for each type of allocation
for e=1:E
    #@constraint(JM,sum(sum(U[:,:,e]))<=tmax[e])
    #@constraint(JM,sum(sum(U[:,:,e]))>=tmin[e])
end
# maximum and minimum no of allocations at each timestep for each allocation

for e=1:E
    for t=1:T
        @constraint(JM,sum(U[:,t,e])<=t1max[e])
        @constraint(JM,sum(U[:,t,e])>=t1min[e])
    end
end

In [ ]:
# regularization distance is positive
#@constraint(JM,w .>= 0)
#@constraint(JM,U .>= 0)
# maximize distance between same type allocations
for e=1:E
    for t=1:T
        #@constraint(JM,U[:,t,e]'*dist_matrix*U[:,t,e]<=w[t,e])  # calculate regularization term
    end
end

In [ ]:
status_1=optimize!(JM) # run the optimization

In [ ]:
# for i=1:289
#     if value.(U[i,2,1]) ==1
#         println("paid")
#     elseif value.(U[i,2,2]) ==1
#         println("cv")
#     else
#         println("bus")
#     end
# end

In [ ]:
U_final=value.(U);  # MIP solution
dfU = DataFrame(curbnum=Int[], curbname=String[], Time=Int[], Parkingtype=String[], loc_start=String[], loc_end=String[])
for t=1:T
    mp=0;
    for i=1:nb
        for j=1:curb_data[i,3]
            mp=mp+1;
            time = (start_time + t - 1) * 15
            if U_final[mp,t,1]==1
                push!(dfU,[mp, string(curb_data[i,2]), time, "Paid", string(curb_data[i,7]), string(curb_data[i,8])])
            elseif U_final[mp,t,2]==1
                push!(dfU,[mp, string(curb_data[i,2]), time, "CV", string(curb_data[i,7]), string(curb_data[i,8])])
            elseif U_final[mp,t,3]==1
                push!(dfU,[mp, string(curb_data[i,2]), time, "Bus", string(curb_data[i,7]), string(curb_data[i,8])])
            end
        end
    end
end

In [ ]:
#dfU=DataFrame(Any[U_final[:,i,j] for j=1:E for i=1:T],:auto)
CSV.write("U_final.csv",dfU)
# dfPaid=DataFrame(curb_value_paid_tot[:,50:59],:auto)
# CSV.write("paid_objective.csv",dfPaid)
# dfCV=DataFrame(curb_value_cv_tot[:,50:59],:auto)
# CSV.write("cv_objective.csv",dfCV)
# dfBus=DataFrame(curb_value_bus_tot[:,50:59],:auto)
# CSV.write("bus_objective.csv",dfBus)

In [ ]:
println("MIP optimal solution")
println(sum(sum(curb_value_paid_tot[:,start_time+1:start_time+T].*value.(U[:,:,1])))+sum(sum(curb_value_cv_tot[:,start_time+1:start_time+T].*value.(U[:,:,2])))+sum(sum(curb_value_bus_tot[:,start_time+1:start_time+T].*value.(U[:,:,3]))))
MIP_sol_opt=sum(sum(curb_value_paid_tot[:,start_time+1:start_time+T].*value.(U[:,:,1])))+sum(sum(curb_value_cv_tot[:,start_time+1:start_time+T].*value.(U[:,:,2])))+sum(sum(curb_value_bus_tot[:,start_time+1:start_time+T].*value.(U[:,:,3])))

In [ ]:
using JSON
mutable struct JTT
  curbnum::Int64
  curbname::String
  Time::Int64
  Parkingtype::String
  loc_start::String
  loc_end::String
end
jt = []
for r in eachrow(dfU)
    push!(jt, JTT(r.curbnum, r.curbname, r.Time, r.Parkingtype, r.loc_start, r.loc_end))
end
open("U_final.json", "w") do f
    write(f, JSON.json(jt))
end